# Git Branch and Folder Structure

In [1]:
from IPython.display import display, HTML

display(HTML(data="""
<style>
    div#notebook-container    { width: 95%; }
    div#menubar-container     { width: 65%; }
    div#maintoolbar-container { width: 99%; }a
</style>
"""))

In [2]:
!pip install selenium beautifulsoup4 pandas

# Exercise 2

In [3]:
#2. Task Description
#Scrape all Data Science job offers from the Bumeran platform that match the following filters (using code not by hand!):

# this library is to manipulate browser
from selenium import webdriver

# We call ChromeDriver
from webdriver_manager.chrome import ChromeDriverManager

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup

import time
import re

In [4]:
# driver = webdriver.Chrome( ChromeDriverManager().install() )
# Maximize window
driver = webdriver.Chrome()
driver.maximize_window()
driver.get("https://www.bumeran.com.pe/empleos.html")

wait = WebDriverWait(driver, 10)

# Click en "Fecha de publicación"
fecha_btn = wait.until(EC.element_to_be_clickable((By.XPATH, "//button[contains(., 'Fecha de publicación')]")))
fecha_btn.click()

# Click en "Menor a 15 días" - Texto exacto
opcion = wait.until(EC.element_to_be_clickable((By.XPATH, "//*[text()='Menor a 15 días']")))
opcion.click()

# Click en "Área" (esto abre el buscador)
area_btn = wait.until(EC.element_to_be_clickable((By.XPATH, "//button[contains(., 'Área')]")))
area_btn.click()
opcion = wait.until(EC.element_to_be_clickable((By.XPATH, "//*[text()='Tecnología, Sistemas y Telecomunicaciones']")))
opcion.click()

# Click en "Subárea"
subarea_btn = wait.until(EC.element_to_be_clickable((By.XPATH, "//button[contains(., 'Subárea')]")))
subarea_btn.click()

# Click en "Programación"
programacion_option = wait.until(EC.element_to_be_clickable((By.XPATH, "//*[text()='Programación']")))
programacion_option.click()

# Escribir "Departamento"
depto_input = wait.until(EC.element_to_be_clickable((By.XPATH, "//button[contains(., 'Departamento')]")))
depto_input.click()
opcion = wait.until(EC.element_to_be_clickable((By.XPATH, "//*[text()='Lima']")))
opcion.click()

# Esperar a que se carguen los resultados filtrados
wait.until(EC.presence_of_element_located((By.XPATH, "//a[.//h2]")))

# Capturar el HTML filtrado en una variable
html_filtrado = driver.page_source

print("HTML capturado y almacenado en la variable 'html_filtrado'")
print(f"Tamaño del HTML: {len(html_filtrado)} caracteres")

HTML capturado y almacenado en la variable 'html_filtrado'
Tamaño del HTML: 261820 caracteres


In [5]:
soup = BeautifulSoup(html_filtrado, "html.parser")

# Buscar elementos de paginación (ajusta el selector según la estructura real)
# Posibles selectores para Bumeran:
pages_elements = soup.select("a[href*='page=']") or soup.select(".pagination a") or soup.select("nav a[href*='page']")

if pages_elements:
    # Extraer números de página
    pages = []
    for element in pages_elements:
        text = element.get_text().strip()
        if text.isdigit():
            pages.append(int(text))
    
    max_page = max(pages) if pages else 1
    print(f"Páginas encontradas: {pages}")
    print(f"Total de páginas: {max_page}")
else:
    print("No se encontraron elementos de paginación")
    max_page = 1

Páginas encontradas: [2, 3, 4, 5, 6]
Total de páginas: 6


In [6]:
#Ver la URL actual completa
current_url = driver.current_url
print(f"URL actual: {current_url}")

URL actual: https://www.bumeran.com.pe/en-lima/empleos-area-tecnologia-sistemas-y-telecomunicaciones-subarea-programacion-publicacion-menor-a-15-dias.html


In [7]:
# Ahora recolectar links de todas las páginas
job_links = set()

for page in range(1, int(max_page) + 1):
    if page == 1:
        print(f"Scraping página {page}/{max_page} (HTML filtrado)...")
        soup = BeautifulSoup(html_filtrado, "html.parser")
    else:
        # Construir URL exactamente como en tu ejemplo
        url = f"{current_url}?page={page}"
        
        print(f"Scraping página {page}/{max_page}")
        print(f"URL: {url}")
        driver.get(url)
        time.sleep(8)
        soup = BeautifulSoup(driver.page_source, "html.parser")
        
        # DEBUG: Ver si hay ofertas
        ofertas_en_pagina = soup.select("a[href*='/empleos/']")
        print(f"  Ofertas encontradas en HTML: {len(ofertas_en_pagina)}")
    
    # Contar enlaces antes
    initial_count = len(job_links)
    
    # Buscar ofertas
    for a in soup.select("a[href*='/empleos/']"):
        link = a.get("href")
        if link and link.startswith("/"):
            link = "https://www.bumeran.com.pe" + link
            job_links.add(link)
    
    new_links = len(job_links) - initial_count
    print(f"  Enlaces agregados en página {page}: {new_links}")

print(f"Total de ofertas recolectadas: {len(job_links)}")

Scraping página 1/6 (HTML filtrado)...
  Enlaces agregados en página 1: 20
Scraping página 2/6
URL: https://www.bumeran.com.pe/en-lima/empleos-area-tecnologia-sistemas-y-telecomunicaciones-subarea-programacion-publicacion-menor-a-15-dias.html?page=2
  Ofertas encontradas en HTML: 20
  Enlaces agregados en página 2: 20
Scraping página 3/6
URL: https://www.bumeran.com.pe/en-lima/empleos-area-tecnologia-sistemas-y-telecomunicaciones-subarea-programacion-publicacion-menor-a-15-dias.html?page=3
  Ofertas encontradas en HTML: 20
  Enlaces agregados en página 3: 20
Scraping página 4/6
URL: https://www.bumeran.com.pe/en-lima/empleos-area-tecnologia-sistemas-y-telecomunicaciones-subarea-programacion-publicacion-menor-a-15-dias.html?page=4
  Ofertas encontradas en HTML: 20
  Enlaces agregados en página 4: 20
Scraping página 5/6
URL: https://www.bumeran.com.pe/en-lima/empleos-area-tecnologia-sistemas-y-telecomunicaciones-subarea-programacion-publicacion-menor-a-15-dias.html?page=5
  Ofertas encon

In [10]:
# SCRAPEAR EL DETALLE DE LAS OFERTAS
import random

def extract_job_details_stealth(driver, job_url):
    """Extraer detalles completos con técnicas anti-detección"""
    try:
        # Pausa aleatoria entre 3-6 segundos para simular comportamiento humano
        wait_time = random.uniform(3, 6)
        print(f"   ⏳ Esperando {wait_time:.1f}s antes de acceder...")
        time.sleep(wait_time)
        
        # Navegar a la página
        driver.get(job_url)
        
        # Esperar más tiempo para que cargue completamente
        time.sleep(random.uniform(3, 8))
        
        page_source = driver.page_source
        soup = BeautifulSoup(page_source, "html.parser")
        
        # 1. EXTRAER TÍTULO
        titulo = None
        title_selectors = [
            "h1.sc-iHrGRV.gplQEj",  # Selector específico que encontraste
            "h1.sc-iHrGRV",         # Versión más general
            "h1[class*='sc-iHrGRV']", # Con wildcards
            "h1",                    # Fallback genérico
            "[data-testid='job-title']",
            ".job-title",
            "h1[class*='title']"
        ]
        
        for selector in title_selectors:
            try:
                title_element = soup.select_one(selector)
                if title_element and title_element.get_text(strip=True):
                    titulo = title_element.get_text(strip=True)
                    break
            except:
                continue
        
        # 2. EXTRAER EMPRESA
        empresa = None
        company_selectors = [
            "div.sc-lbihag.edYXlV",         # Selector específico que encontraste
            "div.sc-lbihag",                # Versión más general
            "div[class*='sc-lbihag']",      # Con wildcard
            "a[href*='/perfiles/empresa'] div.sc-lbihag", # Más específico con el contexto
            "a[href*='/perfiles/empresa'] div",  # Fallback dentro del link de empresa
            "h2[class*='sc-']",             # Patrón similar al título
            "h2",                           # Fallback genérico
            "[data-testid='company-name']",
            ".company-name",
            "span[class*='company']"
        ]
        
        for selector in company_selectors:
            try:
                company_element = soup.select_one(selector)
                if company_element and company_element.get_text(strip=True):
                    empresa = company_element.get_text(strip=True)
                    break
            except:
                continue
        
        # 3. EXTRAER UBICACIÓN
        ubicacion = None
        location_selectors = [
            "[data-testid='job-location']",
            ".location",
            "span[class*='location']",
            "div[class*='location']",
            "*[class*='ubica']",     # Para "ubicación" en español
            "span:contains('Lima')", # Específico para Lima
        ]
        
        for selector in location_selectors:
            try:
                if ":contains(" in selector:
                    # Usar find() para selectores con :contains
                    location_element = soup.find(string=lambda text: text and 'Lima' in text)
                    if location_element:
                        ubicacion = location_element.strip()
                        break
                else:
                    location_element = soup.select_one(selector)
                    if location_element and location_element.get_text(strip=True):
                        ubicacion = location_element.get_text(strip=True)
                        break
            except:
                continue
        
        # 4. EXTRAER DESCRIPCIÓN
        descripcion = None
        description_selectors = [
            "[data-testid='job-description']",
            ".job-description",
            "div[class*='description']",
            "div[class*='detail']",
            "section[class*='description']",
            ".content",
            "div[class*='sc-'][class*='content']"
        ]
        
        for selector in description_selectors:
            try:
                desc_element = soup.select_one(selector)
                if desc_element:
                    descripcion = desc_element.get_text(strip=True)[:500]  # Limitar a 500 chars
                    break
            except:
                continue
        
        # 5. EXTRAER SALARIO (si disponible)
        salario = None
        salary_selectors = [
            "[data-testid='salary']",
            ".salary",
            "span[class*='salary']",
            "div[class*='salary']",
            "*:contains('S/')",  # Soles peruanos
            "*:contains('USD')",
            "*:contains('sueldo')"
        ]
        
        for selector in salary_selectors:
            try:
                if ":contains(" in selector:
                    # Buscar texto que contenga símbolos de moneda
                    salary_element = soup.find(string=lambda text: text and ('S/' in text or 'USD' in text or 'sueldo' in text.lower()))
                    if salary_element:
                        salario = salary_element.strip()
                        break
                else:
                    salary_element = soup.select_one(selector)
                    if salary_element and salary_element.get_text(strip=True):
                        salario = salary_element.get_text(strip=True)
                        break
            except:
                continue
        
        # 6. EXTRAER MODALIDAD (Remoto, Presencial, Híbrido)
        modalidad = None
        modality_selectors = [
            "*:contains('remoto')",
            "*:contains('presencial')",
            "*:contains('híbrido')",
            "*:contains('home office')",
            ".work-mode",
            "[data-testid='work-mode']"
        ]
        
        for selector in modality_selectors:
            try:
                if ":contains(" in selector:
                    modality_text = soup.find(string=lambda text: text and any(word in text.lower() for word in ['remoto', 'presencial', 'híbrido', 'home office']))
                    if modality_text:
                        modalidad = modality_text.strip()
                        break
                else:
                    modality_element = soup.select_one(selector)
                    if modality_element:
                        modalidad = modality_element.get_text(strip=True)
                        break
            except:
                continue
        
        # Log de éxito
        if titulo:
            print(f"   ✅ Título: {titulo[:50]}...")
        
        return {
            "URL": job_url,
            "Titulo": titulo or "No encontrado",
            "Empresa": empresa or "No encontrado", 
            "Ubicacion": ubicacion or "No encontrado",
            "Descripcion": descripcion or "No encontrado",
            "Salario": salario or "No especificado",
            "Modalidad": modalidad or "No especificado"
        }
    
    except Exception as e:
        print(f"   ❌ Error procesando: {str(e)}")
        return {
            "URL": job_url,
            "Titulo": f"Error: {str(e)}",
            "Empresa": "Error",
            "Ubicacion": "Error", 
            "Descripcion": "Error",
            "Salario": "Error",
            "Modalidad": "Error"
        }

In [12]:
# EJECUTAR EXTRACCIÓN DE DETALLE
print("Iniciando extracción de detalles...")
jobs = []

for idx, job_url in enumerate(job_links, start=1):
    print(f"\n[{idx}/{len(job_links)}] Procesando: {job_url.split('/')[-1]}")
    
    job_details = extract_job_details_stealth(driver, job_url)
    jobs.append(job_details)
    
    # Mostrar progreso cada 10
    if idx % 10 == 0:
        successful = len([j for j in jobs if j['Titulo'] and not j['Titulo'].startswith('Error')])
        print(f"\n  PROGRESO: {idx}/{len(job_links)} | Exitosas: {successful}/{idx}")
    
    # Pausa entre requests para evitar ser bloqueado
    if idx < len(job_links):  # No esperar después del último
        time.sleep(random.uniform(1, 3))

print(f"\n🧪 PRUEBA COMPLETADA")
print(f"📊 Total procesadas: {len(jobs)} ofertas")
successful_jobs = len([j for j in jobs if j['Titulo'] and not j['Titulo'].startswith('Error')])
print(f"✅ Exitosas: {successful_jobs}")
print(f"❌ Con errores: {len(jobs) - successful_jobs}")

Iniciando extracción de detalles...

[1/109] Procesando: desarrollador-fullstack-lenguaje-go-y-react-native-software-enterprise-services-s.a.c.-1117949402.html
   ⏳ Esperando 3.2s antes de acceder...
   ✅ Título: Desarrollador FullStack (Lenguaje Go y React Nativ...

[2/109] Procesando: backend-java-sector-bancario-experis-peru-1117965923.html
   ⏳ Esperando 5.3s antes de acceder...
   ✅ Título: Backend Java - Sector bancario...

[3/109] Procesando: desarrollador-.net-senior-fractal-soluciones-it-1117973977.html
   ⏳ Esperando 4.1s antes de acceder...
   ✅ Título: Desarrollador .NET Senior...

[4/109] Procesando: desarrollador-frontend-react-por-proyecto-indra-peru-1117965904.html
   ⏳ Esperando 5.1s antes de acceder...
   ✅ Título: Desarrollador Frontend React - Por Proyecto...

[5/109] Procesando: programador-full-stack-c-exp-en-ia-valtx-1117973381.html
   ⏳ Esperando 6.0s antes de acceder...
   ✅ Título: Programador Full Stack c/ exp en IA...

[6/109] Procesando: analista-programado

   ✅ Título: Practicante de Digitalización...

[47/109] Procesando: analista-desarrollador-java-manpowergroup-peru-1117974662.html
   ⏳ Esperando 5.0s antes de acceder...
   ✅ Título: Analista Desarrollador Java...

[48/109] Procesando: senior-react-developer-encora-1117979242.html
   ⏳ Esperando 4.0s antes de acceder...
   ✅ Título: Senior React Developer...

[49/109] Procesando: programador-ologgi-s.a.c-1117973857.html
   ⏳ Esperando 4.2s antes de acceder...
   ✅ Título: Programador...

[50/109] Procesando: desarrollador-backend-java-inetum-peru-1117961759.html
   ⏳ Esperando 4.9s antes de acceder...
   ✅ Título: Desarrollador Backend Java...

  PROGRESO: 50/109 | Exitosas: 50/50

[51/109] Procesando: desarrollador-full-stack-senior--ecommerce-magento-perfumerias-unidas-1117965886.html
   ⏳ Esperando 5.3s antes de acceder...
   ✅ Título: Desarrollador Full Stack Senior – Ecommerce (Magen...

[52/109] Procesando: analista-programador-.net-|-visual-basic-entelgy-1117975151.html
   ⏳ Es

   ✅ Título: Analista Programador Postgre/Oracle/Híbrido...

[94/109] Procesando: trainee-programador-jr-building-software-1117968407.html
   ⏳ Esperando 5.1s antes de acceder...
   ✅ Título: Trainee Programador Jr...

[95/109] Procesando: tecnico-desarrollador-de-sistemas-1117973188.html
   ⏳ Esperando 5.4s antes de acceder...
   ✅ Título: TÉCNICO DESARROLLADOR DE SISTEMAS...

[96/109] Procesando: mid-node-aws-engineer-encora-1117969139.html
   ⏳ Esperando 5.8s antes de acceder...
   ✅ Título: Mid Node AWS Engineer...

[97/109] Procesando: desarrollador-frontend-senior-fractal-soluciones-it-1117974205.html
   ⏳ Esperando 4.9s antes de acceder...
   ✅ Título: Desarrollador FrontEnd Senior...

[98/109] Procesando: analista-programador-sede-ate-bumeran-selecta-1117969140.html
   ⏳ Esperando 5.6s antes de acceder...
   ✅ Título: Analista Programador - Sede ATE...

[99/109] Procesando: soporte-jr-y-sr-de-plataforma-bmc-helix-remedy-hitss-peru-1117968539.html
   ⏳ Esperando 3.5s antes de ac

In [13]:
# ALMACENAR EL DETALLE
import pandas as pd

# Crear DataFrame
df = pd.DataFrame(jobs)
df.to_csv("bumeran_jobs_TAREA2.csv", index=False, encoding="utf-8-sig")